# Линейная алгебра в NumPy

Не забудьте отправить решения задач в систему Яндекс.Контест:
- [Контест](https://contest.yandex.ru/contest/75563/enter) для 413 группы;
- [Контест](https://contest.yandex.ru/contest/75564/enter) для 414 группы;
- [Контест](https://contest.yandex.ru/contest/75567/enter) для 415 группы;
- [Контест](https://contest.yandex.ru/contest/75567/enter) для 416 группы;

Очень часто различные научные вычисления и алгоритмы связаны с операциями линейной алгебры: перемножением матриц и векторов, вычислением обратных матриц и собственных значений матриц, осуществлением различных разложений матриц и т.д. Все эти операции играют важную роль, например, в методах оптимизации или в машинном обучении. Именно поэтому основные алгоритмы линейной алгебры были реализованы в виде специального подмодуля библиотеки NumPy - `linalg` (сокращение от *linear algebra*). В сегодняшнем семинаре мы посмотрим на некоторый функционал этого модуля.

**Необходимые импорты:**

In [1]:
import numpy as np

Для воспроизводимости результатов, зафиксируем random-seed:

In [2]:
np.random.seed(42)

## Объединение и разбиение массивов

Прежде чем перейти к изучению подмодуля `linalg`, рассмотрим пару полезных функций для объединения и разделения массивов. Эти функции могут оказаться полезными по ряду причин. В качестве яркого примера таких причин можно назвать оптимизацию вычислений. В случае, когда нам необходимо применить некоторую операцию к нескольким массивам, мы можем объединить исходные массивы в один большой массив, чтобы векторизовать вычисления и избежать использования медлительного цикла `for`. После применения желаемой операции к полученному массиву мы можем разбить итоговый массив на несколько массивов, которые будут являться результатом вычислений, примененных к соответствующим входным массивам. Также операции объединения массивов могут быть полезны непосредственно при решении задач линейной алгебры. Например, решая систему линейных уравнений, с помощью функций объединения массивов, вы сможете составить расширенную матрицу системы.

### append

В отличие от списков в Python, массивы в NumPy не определяют методы для добавления новых элементов в конец. Для того, чтобы добавить новые данные в конец массива NumPy придется использовать отдельную функций - `np.append`. В качестве параметров `np.append` принимает массив, в конец которого необходимо дописать данные, массив, который используется в качестве источника данных для дописывания, а также необязательный параметр `axis`, с которым мы познакомились на прошлом семинаре. Рассмотрим примеры использования `np.append`:

In [3]:
array_1d = np.random.randint(-10, 10, size=2)
array_2d = np.random.randint(-10, 10, size=(2, 2))

print(
    f"array_1d:\n{array_1d}",
    f"array_2d:\n{array_2d}",
    f"1D: append 1D:\n{np.append(array_1d, array_1d)}",
    f"1D: append 2D:\n{np.append(array_1d, array_2d)}",
    f"2D: append rows:\n{np.append(array_2d, array_2d, axis=0)}",
    f"2D: append cols:\n{np.append(array_2d, array_1d[..., np.newaxis], axis=1)}",
    sep="\n\n",
)

array_1d:
[-4  9]

array_2d:
[[ 4  0]
 [-3 -4]]

1D: append 1D:
[-4  9 -4  9]

1D: append 2D:
[-4  9  4  0 -3 -4]

2D: append rows:
[[ 4  0]
 [-3 -4]
 [ 4  0]
 [-3 -4]]

2D: append cols:
[[ 4  0 -4]
 [-3 -4  9]]


На приведенных примерах можно пронаблюдать особенности функции `np.append`. Первая особенность, составляющая ключевое отличие от аналогичного метода списков в "чистом" Python, заключается в том, что `np.append` не применяется на месте. Т.е. в результате выполнения функции входные данные не модифицируются, а результат выполнения - новый объект в памяти.

In [4]:
array_appended = np.append(array_1d, array_1d)

print(
    f"array_1d:\n{array_1d}",
    f"array_appended:\n{array_appended}",
    f"is the same: {array_1d is array_appended}",
    sep="\n\n",
)

array_1d:
[-4  9]

array_appended:
[-4  9 -4  9]

is the same: False


Следующая особенность состоит в возможности использования многомерных массивов и в качестве источника данных, и в качестве расширяемого массива. В случае если расширяемый массив является одномерным массивом, в результате выполнения функции все данные из второго массива будут дописаны в конец расширяемого массива. В этом случае поведение `np.append` можно сравнить с поведением метода списков `extend`.

Если же переданные массивы многомерные, их размерности должны быть равными. Т.е. значения атрибутов `ndim` должны быть равны. Иначе `np.append` возбудит исключение. В случае использования многомерных массивов вы также получаете возможность использовать параметра `axis`, чтобы выбирать измерение, вдоль которого будут добавлены данные. По умолчанию массивы будут вытянуты в одномерный массив и объединены.

In [5]:
print(np.append(array_2d, array_2d))

[ 4  0 -3 -4  4  0 -3 -4]


### concatenate

Следующая функция, позволяющая объединять массивы - `np.concatenate`. В отличие от `np.append`, `np.concatenate` позволяет объединить произвольное число массивов и при это явно задать порядок их объединения. В случае многомерных массивов размерности объединяемых массивов должны быть равны, иначе функция возбудит исключение. `np.concatenate`, также как и `np.append`, не модифицирует исходные данные. Однако в отличие от `np.append` `np.concatenate` не всегда создает новый объект в памяти: при передаче параметра `out` `np.concatenate` будет записывать результат в уже существующий в памяти объект.

In [6]:
array_1d = np.random.randint(-10, 10, size=2)
array_2d = np.random.randint(-10, 10, size=(2, 2))

print(
    f"array_1d:\n{array_1d}",
    f"array_2d:\n{array_2d}",
    f"concatenate no axis:\n"
    f"{np.concatenate((array_1d, array_2d), axis=None)}",
    f"concatenate with axis:\n"
    f"{np.concatenate((array_2d, array_1d[np.newaxis, ...]), axis=0)}",
    sep="\n\n",
)

array_1d:
[8 0]

array_2d:
[[ 0 -7]
 [-3 -8]]

concatenate no axis:
[ 8  0  0 -7 -3 -8]

concatenate with axis:
[[ 0 -7]
 [-3 -8]
 [ 8  0]]


Как вы видите, с помощью добавления новых размерностей и указания аргумента `axis` мы можем регулировать, как именно значения массивов будут объединяться, и какой формы будет результирующий массив. Однако при работе с матрицами нам не нужен такой уровень гибкости. В большинстве случаев нам будет необходимо или приписывать столбцы (`axis=1`), или приписывать строки (`axis=0`). Именно для таких случаев в NumPy были добавлены специализированные функции.

### hstack и vstack

Пара функций `np.hstack` (сокращение от *horizontal stacking*) и `np.vstack` (аналогично, *vertical stacking*) позволяют дописывать строки и столбцы соответственно.

In [7]:
array_1d = np.random.randint(-10, 10, size=2)
array_2d = np.random.randint(-10, 10, size=(2, 2))

print(
    f"array_1d:\n{array_1d}",
    f"array_2d:\n{array_2d}",
    "hstack 1d to 2d:\n"
    f"{np.hstack((array_2d, array_1d[..., np.newaxis]))}",
    "hstack 2d to 1d:\n"
    f"{np.hstack((array_1d[..., np.newaxis], array_2d))}",
    "vstack 1d to 2d:\n"
    f"{np.vstack((array_2d, array_1d))}",
    "vstack 2d to 1d:\n"
    f"{np.vstack((array_1d, array_2d))}",
    sep="\n\n",
)

array_1d:
[-9  1]

array_2d:
[[ -5  -9]
 [-10   1]]

hstack 1d to 2d:
[[ -5  -9  -9]
 [-10   1   1]]

hstack 2d to 1d:
[[ -9  -5  -9]
 [  1 -10   1]]

vstack 1d to 2d:
[[ -5  -9]
 [-10   1]
 [ -9   1]]

vstack 2d to 1d:
[[ -9   1]
 [ -5  -9]
 [-10   1]]


### split

Очевидно, если есть возможность слияние массивов в один, также существует возможность разделения одного массива на несколько массивов. Начнем с первого антипода - антипод функции `np.concatenate` - `np.split`. С помощью данной функции вы можете разбить переданный массив на несколько подмассивов по заданному правилу вдоль заданного направления.

In [8]:
def convert_npsplit_to_string(
    parts: list[np.ndarray],
) -> str:
    return ",\n".join(
        [str(part) for part in parts]
    )

In [9]:
array = np.random.randint(-10, 10, size=(6, 4))

print(
    f"array:\n{array}",
    "split-use-int-rule:\n"
    f"{convert_npsplit_to_string(np.split(array, 3))}",
    "split-use-slice-rule:\n"
    f"{convert_npsplit_to_string(np.split(array, [1, 4]))}",
    "split-use-axis:\n"
    f"{convert_npsplit_to_string(np.split(array, 2, axis=1))}",
    sep="\n\n",
)

array:
[[ 1  6 -1  5]
 [ 4  4  8  1]
 [ 9 -8 -6  8]
 [-4 -2 -4  7]
 [-7  3  7 -2]
 [-9  9  4 -4]]

split-use-int-rule:
[[ 1  6 -1  5]
 [ 4  4  8  1]],
[[ 9 -8 -6  8]
 [-4 -2 -4  7]],
[[-7  3  7 -2]
 [-9  9  4 -4]]

split-use-slice-rule:
[[ 1  6 -1  5]],
[[ 4  4  8  1]
 [ 9 -8 -6  8]
 [-4 -2 -4  7]],
[[-7  3  7 -2]
 [-9  9  4 -4]]

split-use-axis:
[[ 1  6]
 [ 4  4]
 [ 9 -8]
 [-4 -2]
 [-7  3]
 [-9  9]],
[[-1  5]
 [ 8  1]
 [-6  8]
 [-4  7]
 [ 7 -2]
 [ 4 -4]]


### hsplit и vsplit

Также аналоги-разделители определены и для функций `np.hstack` и `np.vstack` - `np.hsplit` и `np.vsplit`. Обе эти функции аналогичны функции `np.split` с фиксированным значением аргумента `axis`: `axis=0` для `vsplit`, `axis=1` для `hsplit`.

In [10]:
array = np.random.randint(-10, 10, size=(6, 4))

print(
    f"array:\n{array}",
    "vsplit:\n"
    f"{convert_npsplit_to_string(np.vsplit(array, 3))}",
    "hsplit:\n"
    f"{convert_npsplit_to_string(np.hsplit(array, [1, 3]))}",
    sep="\n\n",
)

array:
[[ 1 -3  4 -8]
 [ 3  6 -7  7]
 [-3 -7 -9 -5]
 [-1 -7  7  1]
 [-9 -1 -7  3]
 [ 5  4 -3  3]]

vsplit:
[[ 1 -3  4 -8]
 [ 3  6 -7  7]],
[[-3 -7 -9 -5]
 [-1 -7  7  1]],
[[-9 -1 -7  3]
 [ 5  4 -3  3]]

hsplit:
[[ 1]
 [ 3]
 [-3]
 [-1]
 [-9]
 [ 5]],
[[-3  4]
 [ 6 -7]
 [-7 -9]
 [-7  7]
 [-1 -7]
 [ 4 -3]],
[[-8]
 [ 7]
 [-5]
 [ 1]
 [ 3]
 [ 3]]


## Операции с матрицами

Начнем изучение функционала NumPy для работы с операциями линейной алгебры с операций, доступных без подключения подмодуля `linalg`. А именно - рассмотрим все возможные произведения.

### Оператор @ и скалярные произведения

Начиная с версии 3.5 в Python был добавлен новый оператор - `@`, который в контексте массивов NumPy имеет смысл обычного произведения матриц:

In [11]:
vector1 = np.random.randint(-10, 10, size=3)
vector2 = np.random.randint(-10, 10, size=3)

matrix1 = np.random.randint(-10, 10, size=(4, 3))
matrix2 = np.random.randint(-10, 10, size=(3, 4))

print(
    f"vector1:\n{vector1}",
    f"vector2:\n{vector2}",
    f"matrix1:\n{matrix1}",
    f"matrix2:\n{matrix2}",
    sep="\n\n",
)

vector1:
[-3  5  2]

vector2:
[7 4 2]

matrix1:
[[ -2   4   2]
 [-10  -4  -2]
 [-10   1  -3]
 [  0   8   6]]

matrix2:
[[ -3  -8  -8 -10]
 [ -6  -1  -4  -2]
 [ -4  -2  -3   1]]


In [12]:
print(
    f"vector @ vector:\n{vector1 @ vector2}",
    f"vector1 @ matrix2:\n{vector1 @ matrix2}",
    f"matrix1 @ vector2:\n{matrix1 @ vector2}",
    f"matrix @ matrix:\n{matrix1 @ matrix2}",
    sep="\n\n",
)

vector @ vector:
3

vector1 @ matrix2:
[-29  15  -2  22]

matrix1 @ vector2:
[  6 -90 -72  44]

matrix @ matrix:
[[-26   8  -6  14]
 [ 62  88 102 106]
 [ 36  85  85  95]
 [-72 -20 -50 -10]]


Из данных примеров видно, что одномерный массив задает как вектор-строки, так и вектор-столбцы, а оператор `@` относится к произведению одномерных массивов, как к произведениям вектора-строки на вектор-столбец, т.е. как к обычному скалярному произведению векторов. Но как быть, если мы хотим получить матрицу при выполнении операции `@`? Т.е., как быть, если нам необходимо осуществить произведение вектор-столбца на вектор-строку? Для решения данной проблемы существуют два решения: манипуляция с размерностями и функция `np.outer`. Ниже рассмотрены оба подхода.

In [13]:
vector1 = np.random.randint(-10, 10, size=3)
vector2 = np.random.randint(-10, 10, size=3)

print(
    f"vector1:\n{vector1}",
    f"vector2:\n{vector2}",
    "shape manipulation:\n"
    f"{vector1[..., np.newaxis] @ vector2[np.newaxis, ...]}",
    f"np.outer:\n{np.outer(vector1, vector2)}",
    sep="\n\n",
)

vector1:
[ -9 -10   5]

vector2:
[-6 -8  1]

shape manipulation:
[[ 54  72  -9]
 [ 60  80 -10]
 [-30 -40   5]]

np.outer:
[[ 54  72  -9]
 [ 60  80 -10]
 [-30 -40   5]]


Как и в случае с векторизованными операциями, оператор `@` имеет функциональную форму `np.matmul`, с помощью которой можно указать буфер для записи результата.

### Векторное произведение

Векторное произведение не имеет операторной формы записи, но имеет функциональную реализацию в виде функции `np.cross`. Функция принимает на вход массивы различных размерностей и предоставляет возможность указать измерения массивов, вдоль которых расположены векторы для выполнения вычислений.

In [14]:
vector = np.random.randint(-10, 10, size=3)
pack_of_vectors = np.random.randint(-10, 10, size=(3, 3))

print(
    f"vector:\n{vector}",
    f"pack of vectors:\n{pack_of_vectors}",
    f"cross product:\n{np.cross(vector, pack_of_vectors, axis=-1)}",
    sep="\n\n",
)

vector:
[ -3  -8 -10]

pack of vectors:
[[ -8  -6   4]
 [  3  -8 -10]
 [ -6   3  -4]]

cross product:
[[-92  92 -46]
 [  0 -60  48]
 [ 62  48 -57]]


### Практика 1. Смешанное произведение

Итак, в NumPy определены скалярное и векторное произведения. Но, к сожалению, нет смешанного произведения векторов. Чтобы исправить это упущение, реализуем функцию для вычисления смешанного произведения, заложив туда возможность вычислять смешанное произведение для группы  векторов.

*Входные данные*:
- `vec1` - двумерный или одномерный `np.ndarray` чисел с плавающей точкой - вектор или матрица векторов, соответственно;
- `vec2` - двумерный или одномерный `np.ndarray` чисел с плавающей точкой - вектор или матрица векторов, соответственно;
- `vec3` - двумерный или одномерный `np.ndarray` чисел с плавающей точкой - вектор или матрица векторов, соответственно;

*Выходные данные*:
- Число с плавающей точкой, в случае, если аргументы - одномерные массивы, - смешанное произведение векторов;
- Одномерный массив чисел с плавающей точкой, если аргументы - двумерные массивы, - смешанные произведения векторов;

*Сторонние эффекты*:

В случае если аргументы имеют разные размеры, необходимо возбудить исключение `ShapeMismatchError`. В случае, если хотя бы одна из переданных матриц имеет размерность, большую двух, необходимо возбудить исключение `ShapeMismatchError`.

*Замечания*:
- Гарантируется, что переданные массивы непустые;
- Считаем, что вектора в матрице хранятся вдоль измерения `axis=1`, т.е. вектора - строки матрицы;
- Смешанным произведением векторов $(\vec{a}, \vec{b}, \vec{c})$ называют скалярное произведение векторов $(\vec{a}, \vec{h})$, где вектор $\vec{h} = \vec{b} \times \vec{c}$ - векторное произведение векторов $\vec{b}, \vec{c}$.

**Необходимые импорты**:

In [15]:
from numbers import Real

**Решение**:

In [16]:
class ShapeMismatchError(Exception):
    pass

In [ ]:
def get_mixed_product(
    vec1: np.ndarray,
    vec2: np.ndarray,
    vec3: np.ndarray,
) -> np.ndarray | Real:
    # код семинариста

    return 0

**Тестирование**:

In [18]:
vec1 = np.arange(3)
vec2 = np.array([1, 0, 0])
vec3 = np.array([0, 1, 0])

result = get_mixed_product(vec1, vec2, vec3)
assert result == 2

result = get_mixed_product(
    np.repeat(vec1[np.newaxis, ...], 3, axis=0),
    np.repeat(vec2[np.newaxis, ...], 3, axis=0),
    np.repeat(vec3[np.newaxis, ...], 3, axis=0),
)
assert np.all(result == np.full_like(vec1, fill_value=2))

try:
    get_mixed_product(
        np.repeat(vec1[np.newaxis, ...], 3, axis=0),
        vec2,
        vec3,
    )

except ShapeMismatchError:
    pass

else:
    assert False, "exception expected"

try:
    get_mixed_product(
        vec1[np.newaxis, np.newaxis, ...],
        vec2,
        vec3,
    )

except ShapeMismatchError:
    pass

else:
    assert False, "exception expected"

AssertionError: 

### Вычисление основных характеристик

**Норма**

Во многих задачах полезным бывает вычислить норму вектора, например, для последующей нормировки. Для этой цели стоит использовать функцию `np.linalg.norm`, которая обладает очень широким функционалом и позволяет вычислять как нормы векторов, так и нормы матриц. Причем, с помощью аргумента `ord` вы можете выбрать, какой именно алгоритм вычисления нормы следует использовать. По умолчанию вычисляется, знакомая вам со школы, l2-норма. В общем же случае эта норма носит название Евклидовой нормы или нормы Фробениуса. Также у этой функции есть аргумент `axis`, благодаря которому вы получаете возможность вычисления нормы вдоль указанных измерений массива. 

In [ ]:
matrix = np.random.randint(-10, 10, size=(3, 3))

print(
    f"matrix:\n{matrix}",
    f"matrix norm:\n{np.linalg.norm(matrix)}",
    f"vectors' norms\n{np.linalg.norm(matrix, axis=-1)}",
    "custom vectors' norm:\n"
    f"{np.sqrt(np.sum(matrix ** 2, axis=-1))}",
    sep="\n\n",
)

**Ранг**

Для выполнения ряда операций требуется определение ранга матрицы. Например, определение ранга матрицы может быть необходимым шагом перед обращением матрицы во время решения матричных уравнений. В NumPy определение ранга матрицы осуществляется с помощью функции `np.linalg.matrix_rank`, в основе которой лежит алгоритм сингулярного разложения матриц.

In [ ]:
matrix_full_rank = np.diag(np.random.randint(1, 5, size=4))
matrix_not_full_rank = matrix_full_rank.copy()
matrix_not_full_rank[1, 1] = 0

print(
    f"full-rank matrix:\n{matrix_full_rank}",
    f"not full-rank matrix:\n{matrix_not_full_rank}",
    "full-rank matrix rank:\n"
    f"{np.linalg.matrix_rank(matrix_full_rank)}",
    "not full-rank matrix rank:\n"
    f"{np.linalg.matrix_rank(matrix_not_full_rank)}",
    sep="\n\n",
)

**Детерминант**

Детерминант матрицы является ее очень важной численной характеристикой. Вычисление детерминанта в NumPy возможно с помощью функции `np.linalg.det`. Однако, поскольку вычисление детерминанта связано с вычислениями произведений, в тех случаях, когда матрица является очень большой, а её элементы - это очень большие числа, в функции `np.linalg.det` может произойти переполнение, и посчитанное значение нельзя будет использовать в дальнейших вычислениях. С этой целью в NumPy реализована специальная функция `np.linalg.slogdet`, которая возвращает знак детерминанта и натуральный логарифм его модуля.

In [ ]:
matrix = np.diag(np.random.randint(1, 5, size=4))

print(
    f"matrix:\n{matrix}",
    f"det:\n{np.linalg.det(matrix)}",
    f"slogdet:\n{np.linalg.slogdet(matrix)}",
    sep="\n\n",
)

**Собственные числа**

Собственные числа и собственные векторы матрицы также являются очень важным ее характеристиками. Их вычисление может быть полезно, как для общего анализа некоторого линейного оператора, так и в качества важного шага в каком-либо алгоритме. Например, вычисление собственных чисел является важным шагом при выполнении алгоритма поиска особых точек изображения. Вычисление собственных чисел реализуется с помощью функции `numpy.linalg.eig`.

In [ ]:
matrix = np.diag(np.random.randint(1, 5, size=4))
eigen_values, eigen_vectors = np.linalg.eig(matrix)

print(
    f"matrix:\n{matrix}",
    f"eigen values:\n{eigen_values}",
    f"eigen vectors:\n{eigen_vectors}",
    sep="\n\n",
)

## Системы уравнений

Разумеется, одной из важных частей линейной алгебры являются линейные уравнения. Поэтому часть функционала подмодуля `linalg` посвящена решению именно этой проблемы. В этом блоке мы рассмотрим основные функции, которые могут оказаться полезными при решении практических задач.

**Обращение матрицы**

При решении многих задач возникает необходимость в вычислении обратной матрицы. Такая задача, например, может возникать при вычислении коэффициентов МНК, о чем мы поговорим ниже. В NumPy вычисление обратной матрицы возможно с помощью специальной функции `np.linalg.inv`. 

In [ ]:
matrix = np.random.randint(-10, 10, size=(3, 3))
matrix_inv = np.linalg.inv(matrix)

print(
    f"matrix:\n{matrix}",
    f"matrix inverse:\n{matrix_inv}",
    f"product:\n{np.round(matrix_inv @ matrix, 2)}",
    sep="\n\n",
)

**Решение системы линейных уравнений**

В части случаев имеется возможность получения точного аналитического решения системы линейных уравнений. В этих случаях система линейных уравнений может быть решена с помощью специальной функции `np.linalg.solve`. Аргументами функции являются матрица коэффициентов и столбец правой части.

Для примера рассмотрим решение системы линейных уравнений:

$$
\begin{equation*}
 \begin{cases}
   x_1 + 2x_2 = 1 \\
   3x_1 + 5x_2 = 2
 \end{cases}
\end{equation*}
$$

In [ ]:
equation_matrix = np.array([[1, 2], [3, 5]])
right_part = np.array([1, 2])

solution = np.linalg.solve(equation_matrix, right_part)
print(
    ", ".join([
        f"x{i + 1} = {np.round(solution[i], 2)}"
        for i in range(solution.size)
    ])
)

**МНК**

Стоит отметить, что далеко не всегда существует возможность нахождения точного решения системы линейных уравнений. В реальной задаче может существовать большое количество уравнений, противоречащих друг другу. В таких условиях ищется оптимальное решение системы, в смысле минимизации некоторой ошибки. Собственно, такая постановка задачи очень напоминает нам постановку задачи МНК. На самом деле МНК можно использовать в качестве метода вычисления оптимального решения системы линейных уравнений, а потому этот алгоритм также реализован в NumPy.

В примере ниже мы рассмотрим функционал NumPy для решения классической задачи, в которой используется МНК - восстановление линейной зависимости.

Предположим, что у нас есть несколько тестовых точек, и мы пытаемся по ним восстановить исходную зависимость вида:
$$y = ax + b, $$

где $a$ и $b$ - искомые коэффициенты.

Пусть известны следующие координаты точек:

In [ ]:
abscissa = np.array([0, 1, 2, 3])
ordinates = np.array([-1, 0.2, 0.9, 2.1])

Для использования функции `np.linalg.lstsq` необходимо составить матрицу коэффициентов системы линейных уравнений. Т.е. мы должны свести задачу к решению системы линейных уравнений относительно переменных $a$ и $b$. Обладая указанными выше точками запишем следующую систему:

$$
\begin{equation*}
 \begin{cases}
   ax_1 + b = y_1 \Rightarrow b = -1 \\
   ax_2 + b = y_2 \Rightarrow a + b = 0.2 \\
   ax_3 + b = y_3 \Rightarrow 2a + b = 0.9\\
   ax_4 + b = y_4 \Rightarrow 3a + b = 2.1
 \end{cases}
\end{equation*}
$$

Итак, займемся построением матрицы системы и поиском коэффициентов.

In [ ]:
equation_matrix = np.vstack((abscissa, np.ones_like(abscissa))).T
print(f"exiation_matrix:\n{equation_matrix}", end="\n\n")

coefficients = np.linalg.lstsq(
    equation_matrix, ordinates, rcond=None
)[0]

print(
    f"incline: {np.round(coefficients[0], 2)}",
    f"shift: {np.round(coefficients[1], 2)}",
    sep="\n",
)

## Задача 1. Оптимальное распределение ресурсов

Некоторая компания производит $N$ типов товаров, используя $M$ типов ресурсов. Для учета производственных стоимостей в компании заведена таблица `costs`, которая представляет собой матрицу размеров $M \times N$. Элемент данной матрицы `costs[i][j]` показывает, сколько единиц i-ого ресурса требуется для производства одной единицы j-ого товара. Для учета доступных ресурсов в компании также определен одномерный массив `resource_amounts`, длины $M$. i-ый элемент массива `resource_amounts` соответствует количеству i-ого ресурса, доступного для использования в производстве.

Каждый месяц аналитики компании, на основании данных анализа рынка, озвучивают ожидаемый спрос на производимые товары. Ожидаемый спрос характеризуется одномерным массивов `demand_expected`, состоящим из $N$ элементов. i-ому элементу массива `demand_expected` соответствует число товаров, которое необходимо произвести для удовлетворения ожидаемого спроса.

Ваша задача - разработать функцию, которая позволяет определить, сможет ли компания удовлетворить ожидаемый спрос, учитывая текущее количество ресурсов и текущие производственные стоимости, или нет.

*Входные данные*:
- `costs` - двумерный массив чисел с плавающей точкой - количества ресурсов, требуемых для производства товаров;
- `resource_amounts` - одномерный массив чисел с плавающей точкой - доступное количество ресурсов;
- `demand_expected` - одномерный массив целых чисел - необходимое число товаров, для удовлетворения ожидаемого спроса;

*Выходные данные*:
- Булево значение: `True`, если компания сможет удовлетворить ожидаемый спрос, иначе - `False`.

*Сторонние эффекты*:
- Если размеры входных массивов не согласованы, необходимо возбудить исключение `ShapeMismatchError`;

**Решение**:

In [1]:
import numpy as np


class ShapeMismatchError(Exception):
    pass


def can_satisfy_demand(
    costs: np.ndarray,
    resource_amounts: np.ndarray,
    demand_expected: np.ndarray,
) -> bool:
    if costs.shape[0] != resource_amounts.shape[0] or\
            costs.shape[1] != demand_expected.shape[0]:
        raise ShapeMismatchError('Размеры входных массивов не согласованы')
    need_resorses = costs @ demand_expected
    return np.all((resource_amounts - need_resorses) >= 0)

**Тестирование**:

In [2]:
costs = np.eye(2)


resource_amounts = np.full(shape=2, fill_value=3)

demand_expected = np.full(shape=2, fill_value=2)

costs = np.array([[1, 2],
                  [3, 4]])
resource_amounts = np.array([5, 11])
demand_expected = np.array([1, 2])
expected = True

assert can_satisfy_demand(costs, resource_amounts, demand_expected) == expected

# assert not can_satisfy_demand(costs, demand_expected, resource_amounts)

## Задача 2. Без базиса не выйдет

Дана квадратная матрица $A$. В строках матрицы $A$ записаны векторы N-мерного пространства. Также дан вектор $\vec{a}$. Ваша задача - реализовать функцию, которая бы выполняла следующие вычисления:
- проверяла, являются ли вектора матрицы $A$ базисом в N-мерном пространстве;
- вычисляла бы ортогональные проекции $\vec{a}$ на базисные вектора, а также вычисляла бы ортогональные составляющие каждой проекции, если матрица $A$ задает базис.

*Входные данные*:
- `matrix` - двумерный массив `np.ndarray` - описание потенциального базиса;
- `vector` - одномерный массив `np.ndarray` - вектор, проекции которого необходимо вычислить;

*Выходные данные*:
- Кортеж (`tuple`) двумерных массивов. Первый элемент кортежа - ортогональные проекции вектора на базис, второй элемент - ортогональные составляющие проекций вектора на базис. В случае, если входная матрица не задает базис, элементы кортежа - `None`.

*Сторонние эффекты*:
- Если матрица $A$ не является квадратной, необходимо возбудить исключение `ShapeMismatchError`;
- Если количество столбцов матрицы $A$ отлично от количества элементов входного вектора, необходимо возбудить исключение `ShapeMismatchError`.

*Замечания*:
- Гарантируется, что `vector` - не нулевой вектор.

**Решение**:

In [3]:
def get_projections_components(
    matrix: np.ndarray,
    vector: np.ndarray,
) -> tuple[np.ndarray | None, np.ndarray | None]:
    if (matrix.shape[0] != matrix.shape[1]):
        raise ShapeMismatchError(
            'Матрица matrix должна быть квадратной матрицей')
    if matrix.shape[1] != vector.shape[0]:
        raise ShapeMismatchError(
            'Кол-во столбцов матриц matrix отлично от кол-ва элементов входного вектора vector')
    # Проверяем, что столбцы matrix - базис
    if np.linalg.slogdet(matrix)[0] == 0:
        return None, None
    # Ищем ортогональные проекции
    scalars = matrix @ vector
    basis_norms = np.linalg.norm(matrix, axis=1)**2
    blank = scalars/basis_norms
    projections = np.multiply(matrix, np.reshape(blank, (-1, 1)))
    # Ищем ортогональные проекции ортогональные составляющие проекций вектора на базис
    orthogonals = vector - projections
    return projections, orthogonals

**Тестирование**:

In [4]:
matrix = np.diag([2, 3])
vector = np.arange(start=1, stop=3)
projections_expected = np.array([[1, 0], [0, 2]])
orthogonals_expected = np.array([[0, 2], [1, 0]])

projections, orthogonals = get_projections_components(matrix, vector)

assert np.allclose(projections, projections_expected)
assert np.allclose(orthogonals, orthogonals_expected)

In [5]:
matrix = np.array([[1, 0], [1, 1]])
vector = np.array([0, 1])
projections_expected = np.array([[0, 0], [0.5, 0.5]])
orthogonals_expected = np.array([[0, 1], [-0.5, 0.5]])

projections, orthogonals = get_projections_components(matrix, vector)

assert np.allclose(projections, projections_expected)
assert np.allclose(orthogonals, orthogonals_expected)

In [6]:
matrix = np.array([[1, 2], [2, 4]])
vector = np.array([0, 1])

projections, orthogonals = get_projections_components(matrix, vector)

assert projections is None
assert orthogonals is None

## Задача 3. Адаптивная фильтрация


Фильтрация нужна для того, чтобы отделить полезный сигнал от шума или помех. Например, если вы слушаете музыку, а в фоне есть шум, фильтр может убрать этот шум, чтобы музыка звучала чище.

Адаптивная фильтрация — это более умный способ фильтрации. Она используется, когда шум или сигнал меняются со временем, и обычный фильтр не справляется. Адаптивный фильтр сам подстраивается под изменения, чтобы всегда эффективно убирать шум или выделять нужный сигнал. Например, если вы разговариваете по телефону в шумном месте, адаптивный фильтр будет постоянно подстраиваться, чтобы ваш голос был слышен чётко, даже если шум вокруг меняется.

В данной задаче рассматривается формула адаптивной фильтрации, которая основана на использовании корреляционной матрицы и обучающей выборки для настройки фильтра. Формула имеет вид:

$$ y = R^{-1} V_s $$

где:
- $ y $ — выходной сигнал после фильтрации (матрица размером $ M \times N $)
- $ R^{-1} $ — обратная корреляционная матрица (матрица размером $ M \times M $)
- $ V_s $ — обрабатываемая выборка сигнала (матрица размером $ M \times N $)

Обратная корреляционная матрица $( R^{-1} $) вычисляется следующим образом:

$$ R^{-1} = (I + V_j A V_j^H)^{-1} = I - V_j (I + V_j^H V_j A)^{-1} V_j^H $$

где:
- $ I $ — единичная матрица
- $ V_j $ — обучающая выборка (матрица комплексных чисел размером $ M \times K $)
- $ A $ — диагональная матрица комплексных чисел (размером $ K \times K $), которая определяет мощность подавления
- $ V_j^H $ — матрица, которая получается поэлементным сопряжением матрицы $ V_j $ и транспонированием результата (это называется эрмитово сопряженная матрица) 

*Пояснение к формуле*:
- Корреляционная матрица $ R $ характеризует статистические свойства сигнала и шума. Обратная корреляционная матрица $ R^{-1} $ используется для компенсации корреляции в сигнале, что позволяет выделить полезный сигнал.
- Матрица $ A $ регулирует степень подавления шума или нежелательных компонентов сигнала. Она является диагональной, что упрощает вычисления.
- Обучающая выборка $ V_j $ используется для настройки фильтра на основе известных данных.

**Задача**

Напишите функцию, которая вычисляет выходной сигнал $ y $ по формуле адаптивной фильтрации, используя предоставленные матрицы $ V_s $, $ V_j $ и $ A $.

*Входные данные*:
- `Vs` — матрица обрабатываемой выборки сигнала (тип: `numpy.ndarray`, размерность: $ M \times N $)
- `Vj` — матрица обучающей выборки (тип: `numpy.ndarray`, размерность: $ M \times K $)
- `diag_A` — диагональ матрицы A (тип: `numpy.ndarray`, размерность: $K$)

*Выходные данные*:
- `y` — выходной сигнал после фильтрации (тип: `numpy.ndarray`, размерность: $ M \times N $)

*Сторонние эффекты*:
- Если размеры входных массивов не согласованы, необходимо возбудить исключение `ShapeMismatchError`;


In [8]:
def adaptive_filter(
    Vs: np.ndarray,
    Vj: np.ndarray,
    diag_A: np.ndarray
) -> np.ndarray:
    if Vs.shape[0] != Vj.shape[0] or Vj.shape[1] != diag_A.size:
        raise ShapeMismatchError("Размеры входных данных несогласованны")
    A = np.diag(diag_A)
    I = np.ones((Vj.shape[1], A.shape[1]))
    Vjh = (Vj.conj()).T
    y = Vs - Vj@np.linalg.inv(I + (Vjh@Vj@A))@(Vjh@Vs)

    return y

In [9]:
with open('source/diag_A_data.npy', 'rb') as f:
    diag_A = np.load(f)

with open('source/Vj_data.npy', 'rb') as f:
    Vj = np.load(f)

with open('source/Vs_data.npy', 'rb') as f:
    Vs = np.load(f)

with open('source/y_data.npy', 'rb') as f:
    y_check = np.load(f)

y = adaptive_filter(Vs, Vj, diag_A)
print(np.max(abs(y - y_check)))
print(y[0:2, 0:2], y_check[0:2, 0:2], sep="\n\n")
assert np.allclose(y, y_check)

1.3430364634964696e-05
[[0.51412627-0.33667176j 0.9402776 -1.03916445j]
 [0.84134119-0.12874193j 1.45907399-0.07220803j]]

[[0.51413087-0.33667736j 0.94028227-1.03917j   ]
 [0.84134475-0.1287457j  1.45907768-0.07221181j]]


AssertionError: 